In [12]:
import base64, os
import requests
import json

api_key = os.getenv('API_KEY')

# Path to your image
image_path = "/home/sangyeob/dev/kkp/mukkaebi/apps/fastapi/images/pizza.jpg"

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')
# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

input_data = {
  'image' : base64_image,
  'food_category' : "LUNCH",
  'user_data' : {
    'gender' : '남',
    'age' : 25,
    'height' : 180,
    'weight' : 80,
    'goal' : '다이어트'
  }
}

food_category = json.dumps(input_data['food_category'], ensure_ascii=False)
user_data = json.dumps(input_data['user_data'], ensure_ascii=False)

In [20]:
text = "This meal is "+food_category+". Please perform the following 3 tasks based on the image. \
Task 1 : Identifying and Quantifying Nutritional Values of Foods in the Image. Analyze up to 5 foods \
Task 2 : Summarizing Nutritional Values of All Foods Consumed. \
Task 3 : Evaluate the diet on a scale of A to E based on the balance of nutrients. \
Task 4 : Make a short review depeding on my data : "+user_data+"\
Please answer food in Korean. Answer only in JSON format without explanation. And answer without spaces or line breaks inside the JSON. Use single quotes instead of double quotes inside the JSON and ReviewMessage. \
This is an example. {'task1':[{'떡볶이':{'calories':380,'carbohydrate':86,'protein':10,'fat':4.4}},{'김치':{'calories':34,'carbohydrate':7.1,'protein':2.0,'fat':0.6}},{'콩나물무침':{'calories':45,'carbohydrate':8.6,'protein':3.6,'fat':0.2}},{'부추전':{'calories':174,'carbohydrate':14.2,'protein':5.6,'fat':10.8}},{'잡곡밥':{'calories':210,'carbohydrate':44,'protein':5,'fat':1.5}},{'된장국':{'calories':40,'carbohydrate':5.9,'protein':3.9,'fat':0.5}}],'task2':{'total':{'calories':883,'carbohydrate':165.8,'protein':30.1,'fat':18}},'task3':'E','task4':'ReviewMessage'}\
"

payload = {
  "model": "gpt-4-vision-preview",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": text
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 1000
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

In [23]:
content = response.json()["choices"][0]['message']['content']
content = content.replace("\'", "\"")
content
x = json.loads(content)
x

{'task1': [{'피자': {'calories': 285,
    'carbohydrate': 36,
    'protein': 12,
    'fat': 10}},
  {'불고기피자': {'calories': 300, 'carbohydrate': 30, 'protein': 16, 'fat': 14}},
  {'페퍼로니피자': {'calories': 310, 'carbohydrate': 34, 'protein': 13, 'fat': 15}},
  {'치즈': {'calories': 220, 'carbohydrate': 1, 'protein': 14, 'fat': 17}},
  {'라면': {'calories': 380, 'carbohydrate': 54, 'protein': 9, 'fat': 14}}],
 'task2': {'total': {'calories': 1495,
   'carbohydrate': 155,
   'protein': 64,
   'fat': 70}},
 'task3': 'C',
 'task4': {'ReviewMessage': '25세 남성이시고 목표가 다이어트이신 만큼 과도한 칼로리와 탄수화물, 지방의 섭취는 체중 감량에 도움이 되지 않습니다. 이 식사는 다이어트 목표에 부합하려면 열량을 줄이고, 단백질 섭취를 늘리며, 신선한 채소와 과일을 포함시키는 것이 좋겠습니다. 균형 잡힌 영양소 섭취와 함께 규칙적인 운동도 병행하시길 권장드립니다.'}}